In [6]:
import os
import pandas as pd
import datetime
import re
import logging
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to list all chat files in the directory structure
def list_chat_files(date_directory):
    chat_files = []
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            for team_folder in os.listdir(date_path):
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):
                                    chat_files.append(os.path.join(person_path, file))
    return chat_files

def parse_chat_file(file_path, expected_date_minus_one):
    chat_data = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            message_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*?): (.*)', line)
            system_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*)', line)
            if message_match:
                date_time_str, sender, message = message_match.groups()
            elif system_match:
                date_time_str, info = system_match.groups()
                sender = None
            else:
                continue

            date_time = pd.to_datetime(date_time_str, format='%d/%m/%y, %I:%M %p')

            # Compare parsed date with the date one day before the expected date
            if date_time.date() != expected_date_minus_one:
                continue

            is_person = sender is not None and re.match(r'^[+\d\s-]+$', sender) is None
            chat_data.append((date_time, sender, is_person))
    return chat_data

# Function to create a template dataframe
def create_template_dataframe():
    times = [datetime.datetime(2000, 1, 1, 0, 0) + datetime.timedelta(minutes=1 * i) for i in range(1440)]
    intervals = [time.strftime('%I:%M %p') for time in times]
    df = pd.DataFrame(index=intervals)
    return df

def populate_dataframe(df, parsed_data, start_column_index):
    new_columns = {}  # Dictionary to hold new data before concatenation

    for entry in parsed_data:
        date_time, sender, is_person = entry
        interval_index = min((date_time.hour * 60 + date_time.minute) // 1, 1439)
        interval = df.index[interval_index]

        # Initialize columns in new_columns dictionary if not exist
        if (start_column_index not in new_columns):
            new_columns[start_column_index] = pd.Series(0, index=df.index)
        if (start_column_index + 1 not in new_columns):
            new_columns[start_column_index + 1] = pd.Series(0, index=df.index)

        # Populate the new_columns dictionary
        if is_person:
            new_columns[start_column_index].at[interval] = 1
        else:
            new_columns[start_column_index + 1].at[interval] = 1
    print(f"DataFrame after concatenation: {df.head(1)}")
    # Concatenate new columns to the DataFrame at once
    df = pd.concat([df, pd.DataFrame(new_columns)], axis=1)

    return df, start_column_index + 2

def process_person_chats(chat_files):
    dataframes = {}
    for file in chat_files:
        parts = file.split(os.sep)
        date_folder, person = parts[-4], parts[-2]

        try:
            folder_date = pd.to_datetime(date_folder, format='%Y-%m-%d').date()
        except ValueError:
            continue

        expected_date_minus_one = folder_date - datetime.timedelta(days=1)
        key = f"{folder_date.strftime('%Y-%m-%d')}_{person}"

        if key not in dataframes:
            dataframes[key] = create_template_dataframe()
            start_column_index = 0
        else:
            if not dataframes[key].columns.empty:
                start_column_index = max(dataframes[key].columns) + 1
            else:
                start_column_index = 0

        parsed_data = parse_chat_file(file, expected_date_minus_one)
        dataframes[key], start_column_index = populate_dataframe(dataframes[key], parsed_data, start_column_index)
    print(f"Matrix DataFrame for {key}: {dataframes[key].head()}")  # Debug print
    return dataframes


def count_active_chats_at_time(matrix_df, timestamp):
    # Format the timestamp to match the DataFrame's index format (%I:%M %p)
    interval = timestamp.strftime('%I:%M %p')
    logging.debug(f"Checking active chats at time: {interval}")  # Debugging print

    if interval in matrix_df.index:
        active_chats_count = matrix_df.loc[interval].sum()
        logging.debug(f"Active chats count: {active_chats_count}")  # Debugging print
        return active_chats_count
    else:
        logging.debug("No matching interval found")  # Debugging print
        return 0



# Function to list all chat files in the directory structure
def list_chat_files(date_directory):
    chat_files = []
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            for team_folder in os.listdir(date_path):
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):
                                    chat_files.append(os.path.join(person_path, file))
    return chat_files

def parse_chat_file_for_delay_analysis(file_path):
    # Extract person_name and date from the file path
    person_name = os.path.basename(os.path.dirname(file_path))
    folder_date_str = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(file_path))))
    expected_date_minus_one = pd.to_datetime(folder_date_str, format='%Y-%m-%d').date() - datetime.timedelta(days=1)

    delay_messages = []

    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    last_person_msg_time = None
    last_person_msg_index = None

    for i, line in enumerate(lines):
        message_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*?): (.*)', line)
        if message_match:
            date_time_str, sender, message = message_match.groups()
            date_time = pd.to_datetime(date_time_str, format='%d/%m/%y, %I:%M %p')

            if date_time.date() != expected_date_minus_one:
                continue

            # Check if the sender is a person (not purely numeric or starting with +)
            is_person = sender is not None and not (sender.strip().isnumeric() or sender.startswith('+'))

            logging.info(f"Sender: {sender}, Is Person: {is_person}, Message: {message}")  # Debug line

            if is_person:
                if last_person_msg_time is not None:
                    # Calculate the time difference in minutes
                    time_diff_seconds = (date_time - last_person_msg_time).total_seconds()
                    time_diff_minutes = time_diff_seconds / 60
                    logging.info(f"Time Difference (minutes): {time_diff_minutes}")  # Debug line
                    if time_diff_minutes > 15:
                        # There is a delay
                        delay = True
                    else:
                        # No delay
                        delay = False
                        time_diff_minutes = None
                else:
                    # No previous message from a person
                    delay = False
                    time_diff_minutes = None
                last_person_msg_time = date_time
                last_person_msg_index = i
            else:
                # Not a message from a person
                delay = False
                time_diff_minutes = None

            if delay:
                # Fetch reference messages
                start_index = max(last_person_msg_index - 3, 0)
                end_index = min(i + 4, len(lines) - 1)
                reference_msgs = lines[start_index:end_index + 1]

                delay_messages.append((date_time, sender, message, delay, time_diff_minutes, reference_msgs))

    return delay_messages, file_path


def process_all_files_with_matrix(chat_files, matrix_dataframes):
    all_delay_messages = []

    for file_path in chat_files:
        logging.info(f"Processing file: {file_path}")  # Debug line
        delay_messages, file_path_from_function = parse_chat_file_for_delay_analysis(file_path)
        person_name = os.path.basename(os.path.dirname(file_path))
        matrix_df = matrix_dataframes.get(person_name)

        for message in delay_messages:
            timestamp, sender, msg, delay, time_diff_minutes, reference_msgs = message
            active_chats_count = count_active_chats_at_time(matrix_df, timestamp) if matrix_df is not None else 0
            all_delay_messages.append((timestamp, sender, msg, delay, time_diff_minutes, reference_msgs, active_chats_count))

    return all_delay_messages

def create_delay_data_dataframe_with_active_chats(all_delay_data):
    columns = ['Date', 'Person', 'Chat Group Name', 'Delay', 'Delay in Mins', 'Reference Messages', 'Active Chats at Time of Delay']
    df_data = []

    for delay_instance in all_delay_data:
        date_time, sender, message, delay, time_diff_minutes, reference_msgs, active_chats_count = delay_instance

        file_name = os.path.basename(reference_msgs[0])
        logging.info(f"Processing file name: {file_name}")  # Debugging line

        group_name_match = re.match(r'WhatsApp Chat with (.+?)\.txt', file_name)
        chat_group_name = group_name_match.group(1) if group_name_match else "Unknown Group"

        df_data.append([date_time, sender, chat_group_name, delay, time_diff_minutes, reference_msgs, active_chats_count])

    delay_df = pd.DataFrame(df_data, columns=columns)
    return delay_df

# Main Execution
date_directory = "C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Test\\filtered_chats"
chat_files = list_chat_files(date_directory)
matrix_dataframes = process_person_chats(chat_files)
all_delay_data = process_all_files_with_matrix(chat_files, matrix_dataframes)
delay_analysis_df = create_delay_data_dataframe_with_active_chats(all_delay_data)

# Display the first few rows of the DataFrame
print(delay_analysis_df.head())


DataFrame after concatenation: Empty DataFrame
Columns: []
Index: [12:00 AM]
DataFrame after concatenation:           0  1
12:00 AM  0  0
DataFrame after concatenation:           0  1  2  3
12:00 AM  0  0  0  0
DataFrame after concatenation:           0  1  2  3  4  5
12:00 AM  0  0  0  0  0  0
DataFrame after concatenation:           0  1  2  3  4  5  6  7
12:00 AM  0  0  0  0  0  0  0  0
DataFrame after concatenation:           0  1  2  3  4  5  6  7  8  9
12:00 AM  0  0  0  0  0  0  0  0  0  0
DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   10  11
12:00 AM   0   0   0   0   0   0   0   0   0   0   0   0
DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   10  11  12  13
12:00 AM   0   0   0   0   0   0   0   0   0   0   0   0   0   0
DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15
12:00 AM   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
DataFrame after

DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   ...  76  77  78  79  80  81  \
12:00 AM   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   

          82  83  84  85  
12:00 AM   0   0   0   0  

[1 rows x 86 columns]
DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   ...  78  79  80  81  82  83  \
12:00 AM   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   

          84  85  86  87  
12:00 AM   0   0   0   0  

[1 rows x 88 columns]
DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   ...  80  81  82  83  84  85  \
12:00 AM   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   

          86  87  88  89  
12:00 AM   0   0   0   0  

[1 rows x 90 columns]
DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   ...  82  83  84  85  86  87  \
12:00 AM   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0  

DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  \
12:00 AM   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   

          17  
12:00 AM   0  
DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  \
12:00 AM   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   

          17  18  19  
12:00 AM   0   0   0  
DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   ...  12  13  14  15  16  17  \
12:00 AM   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   

          18  19  20  21  
12:00 AM   0   0   0   0  

[1 rows x 22 columns]
DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   ...  14  15  16  17  18  19  \
12:00 AM   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   

          20  21  22  23  
12:00 AM   0   0   0   0  

[1 rows x 24 columns

DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   ...  72  73  74  75  76  77  \
12:00 AM   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   

          78  79  80  81  
12:00 AM   0   0   0   0  

[1 rows x 82 columns]
DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   ...  74  75  76  77  78  79  \
12:00 AM   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   

          80  81  82  83  
12:00 AM   0   0   0   0  

[1 rows x 84 columns]
DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   ...  76  77  78  79  80  81  \
12:00 AM   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   

          82  83  84  85  
12:00 AM   0   0   0   0  

[1 rows x 86 columns]
DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   ...  78  79  80  81  82  83  \
12:00 AM   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0  

DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   ...  44  45  46  47  48  49  \
12:00 AM   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   

          50  51  52  53  
12:00 AM   0   0   0   0  

[1 rows x 54 columns]
DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   ...  46  47  48  49  50  51  \
12:00 AM   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   

          52  53  54  55  
12:00 AM   0   0   0   0  

[1 rows x 56 columns]
DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   ...  48  49  50  51  52  53  \
12:00 AM   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   

          54  55  56  57  
12:00 AM   0   0   0   0  

[1 rows x 58 columns]
DataFrame after concatenation:           0   1   2   3   4   5   6   7   8   9   ...  50  51  52  53  54  55  \
12:00 AM   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0  

...

2023-11-29 17:03:45,787 - INFO - Processing file name: So shall we start our discussion now?

2023-11-29 17:03:45,787 - INFO - Processing file name: 

2023-11-29 17:03:45,788 - INFO - Processing file name: 23, 12:45 pm - +263 78 377 1666: Yes we can start

2023-11-29 17:03:45,789 - INFO - Processing file name: 

2023-11-29 17:03:45,790 - INFO - Processing file name: 23, 2:57 pm - Sahil Edoofa: How are you doing today ?

2023-11-29 17:03:45,790 - INFO - Processing file name: Are you ready?

2023-11-29 17:03:45,790 - INFO - Processing file name: 23, 1:06 pm - +260 96 6652183: Good afternoon

2023-11-29 17:03:45,790 - INFO - Processing file name: 23, 7:00 pm - Shashwat Edoofa 2: May I know when will Albert be available for the session, please?

2023-11-29 17:03:45,790 - INFO - Processing file name: *B.Sc. in Microbiology* is a 3- year long, full- time course divided into 6 semesters, with each semester lasting for a period of 6 months. Microbiology has emerged as a key biological science 

2023-11-29 17:03:45,830 - INFO - Processing file name: On a positive note, I am here to begin our session

2023-11-29 17:03:45,836 - INFO - Processing file name: 23, 1:04 pm - +263 78 806 5685: Yes

2023-11-29 17:03:45,837 - INFO - Processing file name: 

2023-11-29 17:03:45,837 - INFO - Processing file name: On a positive note, I am here to begin our session

2023-11-29 17:03:45,837 - INFO - Processing file name: 23, 4:47 pm - Shashwat Edoofa 2: May I know whether you are able to fetch your previous term results, please?

2023-11-29 17:03:45,837 - INFO - Processing file name: 23, 7:20 pm - +263 71 878 3872: English literature E

2023-11-29 17:03:45,837 - INFO - Processing file name: 23, 9:57 pm - Shashwat Edoofa 2: Can you please brief me regarding your career aspirations more vividly?

2023-11-29 17:03:45,837 - INFO - Processing file name: 

2023-11-29 17:03:45,837 - INFO - Processing file name: 

2023-11-29 17:03:45,837 - INFO - Processing file name: Are you ready?

2023-11-29 17:03

2023-11-29 17:03:45,872 - INFO - Processing file name: 23, 4:14 pm - +263 77 264 5203: Afternoon sir

2023-11-29 17:03:45,872 - INFO - Processing file name: 23, 5:07 pm - +263 77 264 5203: Thank you sir , l have passion for educating the minors , prior to my profession l still need to further my education

2023-11-29 17:03:45,872 - INFO - Processing file name: 23, 5:23 pm - Shashwat Edoofa 2: Could you kindly shed light on the educational background of your sibling, please?

2023-11-29 17:03:45,872 - INFO - Processing file name: He did his primary and secondary education in Harare. He passed his o level so he is now doing motor mechanics at Harare Poly-technical college.Right now he is on attachment at a local company in Harare. His hobbies are soccer and going to church

2023-11-29 17:03:45,872 - INFO - Processing file name: 23, 7:07 pm - Shashwat Edoofa 2: Thank you for sharing, Pride. I appreciate your sincerity! So, could you kindly tell me how much *Mukaro High School* charges you

2023-11-29 17:03:45,891 - INFO - Processing file name: 23, 3:55 pm - +263 78 833 1400: Edoofa offers a guarantee of admission and scholarship to eligible students who are able to first qualify for the eligibility check. *The enrollment fee includes, assistance with creating a statement of purpose, recommendation letter, multiple university applications, and visa assistance, along with training in the Earn While You Learn Program.* This means students don't have to pay separate fees for multiple applications, SOPs, or recommendations. Edoofa is revolutionizing the admission process for international students.

2023-11-29 17:03:45,891 - INFO - Processing file name: 

2023-11-29 17:03:45,891 - INFO - Processing file name: Bank’s Address: Janakpuri, B Block, New Delhi[DL]

2023-11-29 17:03:45,906 - INFO - Processing file name: 23, 1:27 pm - Shubham Madhwal: @263775906860 I hope you are aware that India is 3 hours 30 minutes ahead of Zimbabwean time as we speak?

2023-11-29 17:03:45,906 - I

                 Date   Person Chat Group Name  Delay  Delay in Mins  \
0 2023-11-27 18:39:00  Arshita   Unknown Group   True          100.0   
1 2023-11-27 19:22:00  Arshita   Unknown Group   True          413.0   
2 2023-11-27 12:09:00  Arshita   Unknown Group   True           74.0   
3 2023-11-27 12:28:00  Arshita   Unknown Group   True           19.0   
4 2023-11-27 19:25:00  Arshita   Unknown Group   True          415.0   

                                  Reference Messages  \
0  [27/11/23, 4:49 pm - Arshita added you\n, 27/1...   
1  [ *A problem is a chance for you to do your be...   
2  [\n, How are you doing?\n, 27/11/23, 12:01 pm ...   
3  [●Physics &\n, ●Technical Graphics & Design \n...   
4  [ *A problem is a chance for you to do your be...   

   Active Chats at Time of Delay  
0                              0  
1                              0  
2                              0  
3                              0  
4                              0  


In [2]:
delay_analysis_df.to_csv("delays.csv", index=False)